# <center> 第二次作业 </center>  
 
## <center> 15200414许赵苏扬</center> 

## 第二次小作业内容  

**1. 作业内容：**  

* 使用多个自变量(两个或两个以上，自行选择)对恋爱情况(因变量)建立logistic回归模型  

**2. 具体要求：**  
    重复课上的workflow，包括:  
		
    1. 模型定义(要求附上基本的文字解释，格式参考附后）  
    
    2. MCMC采样，模型诊断图，后验参数解释  
        * 使用az.plot_trace绘制模型诊断图  
        * 对后验参数结果在模型中的意义进行文字解释  
        * 绘制后验回归模型(使用az.plot_hdi)  

    3. 使用定义好的模型，对新站点的结果进行预测并评估  
        * 按照50-50的分类标准，计算预测结果对应的准确性、敏感性、特异性，并附上简单的文字解释  
    
    4. 对本数据集的预测结果进行评估  
        * 按照50-50的分类标准，计算预测结果对应的准确性、敏感性、特异性，并附上简单的文字解释  
    
    5. 与课上的模型(log_model1:自变量为回避倾向; log_model2:自变量为性别)进行模型比较  

**3. 作业截止时间：12.19**  

**4. 作业提交于和鲸平台**  


* 注1：所有的文字解释仅需增加在notebook中，无需提交额外的文档  

* 注2：上述要求请使用pymc语法实现  

* 注3：在数据预处理时注意缺失值  

* 注4：每个小组内成员所使用的站点数据是相同的，但该作业为个人独立提交，小组内可以互相讨论  

| 组别  | 建立模型使用的站点 | 对新数据进行预测时使用的站点  |  
|---------|------------------------------|-----------------------|  
| 1 | METU   | UCSB  |  
| 2 | Oxford | Poland |  
| 3 | Serbia | VCU |  
| 4 | VCU |Oxford |  
| 5 | UCSB |Serbia |  


**模型定义文字描述格式参考：**  

1. 自变量：xx  

2. 因变量：xx  

3. 数据关系：  

$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1 & {\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 + \beta_1 X_{i1}}}{1 + e^{\beta_0 + \beta_1 X_{i1}}} \\  
\text{priors:} & & \beta_{0}  &  \sim N()  \\  
               & & \beta_1  & \sim N(). \\  
\end{array}  
$$  

# **模型定义：**  

1. 自变量：吸烟情况、焦虑水平  

2. 因变量：恋爱关系  

3. 数据关系：  

# **model5：anxiety、sex~roamntic**  

$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1,\beta_2 & \stackrel{ind}{\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 + \beta_1 X_{i1}+\beta_2 X_{i2}}}{1 + e^{\beta_0 + \beta_1 X_{i1}+ \beta_2 X_{i2}}} \\  
\text{priors:} & & \beta_{0}  &  \sim N\left(0, 0.5^2 \right)  \\  
               & & \beta_1  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_2  & \sim N\left(0, 0.5^2 \right)\\  
\end{array}  
$$

In [184]:
# 导入 pymc 模型包，和 arviz 等分析工具 
import pymc as pm
import arviz as az
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import ipywidgets

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")

In [185]:
# 通过 pd.read_csv 加载数据 Data_Sum_HPP_Multi_Site_Share.csv
df_raw = pd.read_csv('/home/mw/input/bayes20238001/Data_Sum_HPP_Multi_Site_Share.csv')


# 选取VCU站点的数据
df = df_raw[df_raw["Site"] == "VCU"]
# 选取涉及的变量
df = df[["romantic", "anxiety_r", "sex"]]

#重新编码，编码后的数据：1 = "yes"; 2 = "no"
df["romantic"] =  np.where(df['romantic'] == 2, 0, 1)
#0 表示男性，1表示女性
df["sex"] =  np.where(df['sex'] == 1, 0, 1)

#设置索引
df["index"] = range(len(df))
df = df.set_index("index")
coords = {"obs_id": df.index}

In [186]:
with pm.Model() as log_model5:
    log_model5.add_coord('obs_id',df.index, mutable=True)
    anxiety_r = pm.MutableData("anxiety_r", df.anxiety_r, dims="obs_id")
    sex= pm.MutableData("sex", df.sex, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)           #定义beta_2
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * anxiety_r +  beta_2 * sex, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")

In [187]:
# 可视化模型（贝叶斯变量因果图）
pm.model_to_graphviz(log_model5)

In [188]:
#先验预测检验，查看由当前先验组合生成的pi是否都在（0~1）范围内
log5_prior = pm.sample_prior_predictive(samples=50, 
                                          model=log_model5,
                                          random_seed=84735)

Sampling: [beta_0, beta_1, beta_2, y_est]


In [189]:
log5_prior

Inference data with groups:
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [190]:
#MCMC采样&模型诊断
with log_model5:
    # MCMC 近似后验分布
    log_model5_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 28 seconds.


In [191]:
az.plot_trace(log_model5_trace,
              var_names=["beta_0","beta_1","beta_2"],
              figsize=(15,8),
              compact=False)
plt.show()

<Figure size 1500x800 with 6 Axes>

In [192]:
az.summary(log_model5_trace, var_names=["beta_0","beta_1","beta_2"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,0.121,0.264,-0.379,0.619,0.003,0.002,10226.0,12177.0,1.0
beta_1,-0.707,0.177,-1.036,-0.369,0.002,0.001,13255.0,12756.0,1.0
beta_2,0.135,0.290,-0.420,0.662,0.003,0.002,10452.0,11633.0,1.0


# 后验参数解释  

以下的结果显示：  
- $\beta_0 = 0.22$，那么 $e^{\beta_0} = 0.80$， 表明 X1 为 0时，个体恋爱的可能性为 0.80。  
- $\beta_1 = -0.71$， $e^{\beta_1} = 0.49$， 表明焦虑分数每增加1个单位，性别不变的情况下，个体恋爱的发生比变为之前的0.49倍。  
- -$\beta_2 = 0.135$， $e^{\beta_2} = 1.14$， 表明焦虑不变的情况下，男女性别差异为1.14倍  
- 然而，$\beta_0$ 、$\beta_2$ 的94%HDI包括0，说明回避分数不能有效预测恋爱发生的概率。 

In [193]:
# 通过 np.exp 将 beta 参数进行转换
az.plot_posterior(log_model5_trace, var_names=["beta_0","beta_1","beta_2"], transform = np.exp)
plt.show()

<Figure size 2208x552 with 3 Axes>

In [194]:
# 分别获取sex=0及sex=1时，焦虑变量对应的索引值
man_index = xr.DataArray(df.query("sex == 0")["anxiety_r"])["index"].values
femal_index = xr.DataArray(df.query("sex == 1")["anxiety_r"])["index"].values

In [195]:
# 根据上一步获得的索引值，将参数的后验预测分布分成两部分
# 一部分为男性，参数的后验预测分布、焦虑水平、恋爱情况
# 另一部分为女性，参数的后验预测分布、焦虑水平、恋爱情况
man_para = log_model5_trace.sel(obs_id=man_index)
femal_para = log_model5_trace.sel(obs_id=femal_index)


In [196]:
#绘制后验预测回归线
#对于一次抽样，可以绘制出一条曲线，结合循环绘制出50条曲线
#sex=0
for i in range(100):
    sns.lineplot( x = man_para.constant_data["anxiety_r"],
            y = man_para.posterior["pi"].stack(sample=("chain", "draw"))[:,i], 
            c="grey",
            alpha=0.4)
#设置x、y轴标题和总标题    
plt.xlabel("anxiety_r",
           fontsize=12)
plt.ylabel("probability of romantic",
           fontsize=12)
plt.suptitle("100 posterior plausible models_man",
           fontsize=14)

#sex=1         
for i in range(100):
    sns.lineplot( x = femal_para.constant_data["anxiety_r"],
            y = femal_para.posterior["pi"].stack(sample=("chain", "draw"))[:,i], 
            c="orange",
            alpha=0.4)
#设置x、y轴标题和总标题    
plt.xlabel("anxiety_r",
           fontsize=12)
plt.ylabel("probability of romantic",
           fontsize=12)
plt.suptitle("100 posterior plausible models_femal",
           fontsize=14)


sns.despine()
plt.show()

<Figure size 640x480 with 1 Axes>

In [197]:
 #画出sex=0的情况下的后验预测线性模型
az.plot_hdi(
    man_para.constant_data["anxiety_r"],
    man_para.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="C1"
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean1 = man_para.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x =man_para.constant_data["anxiety_r"], 
             y= post_mean1, 
             label="sex=man:posterior mean", 
             color="C1")
#绘制真实数据散点图
sns.scatterplot(
                x = df["anxiety_r"], 
                y= df["romantic"],
                label="observed data", 
                color='Red', 
                alpha=0.5)
#画出sex=1的情况下的后验预测线性模型
az.plot_hdi(
    femal_para.constant_data["anxiety_r"],
    femal_para.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="C1"
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean2 = femal_para.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = femal_para.constant_data["anxiety_r"],
             y= post_mean2, 
             label="sex=femal:posterior mean", 
             color="Green")


#设置图例位置
plt.legend(loc="upper right",
           bbox_to_anchor=(1.5, 1),
           fontsize=12)
sns.despine()

<Figure size 640x480 with 1 Axes>

# **对本数据集的预测结果进行评估**

In [198]:
#组合真实数据与预测数据
log_model5_ppc.posterior_predictive.y_est.stack(sample = ("chain", "draw"))

<xarray.DataArray 'y_est' (obs_id: 151, sample: 20000)>
array([[0, 0, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])
Coordinates:
  * obs_id   (obs_id) int64 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149 150
  * sample   (sample) object MultiIndex
  * chain    (sample) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3 3 3 3 3
  * draw     (sample) int64 0 1 2 3 4 5 6 ... 4993 4994 4995 4996 4997 4998 4999

In [199]:
az.plot_ppc(log_model5_ppc, num_pp_samples=50)

<Axes: xlabel='y_est / y_est'>

<Figure size 640x480 with 1 Axes>

In [200]:
pred_pi = log_model5_ppc.posterior_predictive.y_est.stack(sample = ("chain", "draw")).mean(dim="sample")
pred_pi = pred_pi.to_dataframe()

In [201]:
#将原数据中的X 和Y存入数据框
pred_pi["anxiety_r"] = log_model5_ppc.constant_data.anxiety_r.values
pred_pi["sex"] = log_model5_ppc.constant_data.sex.values
pred_pi["romantic"] = log_model5_ppc.observed_data.y_est.values

#根据分类标准（50-50）生成最终的分类结果
pred_pi["romantic_pred"] = np.where(pred_pi["y_est"] >= 0.5, 1, 0)
pred_pi

,y_est,anxiety_r,sex,romantic,romantic_pred
obs_id,,,,,
0,0.23520,2.132340,1,0,0
1,0.51135,0.295121,1,1,1
2,0.38260,1.064189,1,0,0
3,0.65165,-0.730304,0,1,1
4,0.60255,-0.473948,0,0,1
...,...,...,...,...,...
146,0.50275,0.166943,0,0,1
147,0.37275,1.106915,1,0,0
148,0.68820,-0.773030,1,1,1


In [202]:
confusion_matrix = pd.crosstab(pred_pi["romantic"], pred_pi["romantic_pred"], 
                              rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

Predicted,0,1
Actual,,
0,36,31
1,20,64


In [203]:
true_positive =  confusion_matrix.at[0, 1]
false_positive = confusion_matrix.at[1, 1]
true_negative = confusion_matrix.at[0, 0]
false_negative = confusion_matrix.at[1, 0]

accuracy = (true_positive + false_negative) /(true_positive + false_positive + true_negative + false_negative)
sensitivity = (true_positive) /(true_positive + false_negative)
specificity = (true_negative) / (true_negative + false_positive)

print("准确性:", accuracy)
print("敏感性:", sensitivity)
print("特异性:", specificity)

准确性: 0.33774834437086093
敏感性: 0.6078431372549019
特异性: 0.36


模型对于原数据的预测结果准确性较低，说明对于模型的整体性不高，依恋焦虑和性别对于预测被试是否会恋爱的准确率仅为0.336；敏感性为0.6，说明对于依恋焦虑得分低和性别预测被试进行恋爱的可能性为0.6；特异性为0.36，说明依恋焦虑得分高和性别预测被试不进行恋爱的可能性为0.36.

# **对新数据进行预测、分类、评估**

In [204]:
# 通过 pd.read_csv 加载数据 Data_Sum_HPP_Multi_Site_Share.csv
df_raw = pd.read_csv('/home/mw/input/bayes20238001/Data_Sum_HPP_Multi_Site_Share.csv')


# 选取Oxford站点的数据
df_new = df_raw[df_raw["Site"] == "Oxford"]
# 选取涉及的变量
df_new = df_new[["romantic", "anxiety_r", "sex"]]

#重新编码，编码后的数据：1 = "yes"; 2 = "no"
df_new["romantic"] =  np.where(df_new['romantic'] == 2, 0, 1)
#0 表示男性，1表示女性
df_new["sex"] =  np.where(df_new['sex'] == 1, 0, 1)

#设置索引
df_new["index"] = range(len(df_new))
df_new = df_new.set_index("index")
coords = {"obs_id": df_new.index}

In [205]:
df_new 

,romantic,anxiety_r,sex
index,,,
0,1,-1.975260,0
1,1,-0.323469,0
2,1,-1.172962,1
3,1,0.384442,0
4,0,0.148472,1
...,...,...,...
132,0,0.337248,1
133,1,-0.370663,0
134,1,-1.644902,1


In [207]:
pred_coords ={"obs_id":range(0,137)} 
with log_model5:
    pm.set_data({"anxiety_r":df_new["anxiety_r"],
                "sex": df_new["sex"],
                "y":df_new["romantic"] },
                coords=pred_coords
                )
    prediction = pm.sample_posterior_predictive(log_model5_trace, 
                                                var_names=["y_est"],
                                                predictions=True,
                                                random_seed=84735)
print(df_new)
print(df_new['romantic'].isnull().sum(axis=0))

Sampling: [y_est]


       romantic  anxiety_r  sex
index                          
0             1  -1.975260    0
1             1  -0.323469    0
2             1  -1.172962    1
3             1   0.384442    0
4             0   0.148472    1
...         ...        ...  ...
132           0   0.337248    1
133           1  -0.370663    0
134           1  -1.644902    1
135           0   0.903577    1
136           0   1.894651    1

[137 rows x 3 columns]
0


In [208]:
prediction

Inference data with groups:
	> predictions
	> predictions_constant_data

In [209]:
y_pred = prediction.predictions["y_est"].stack(sample=("chain","draw","obs_id")).values
y_pred_freq = np.bincount(y_pred)/len(y_pred)
bars = plt.bar([0, 1], y_pred_freq, color="#70AD47")
for bar, freq in zip(bars, y_pred_freq):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f"{freq:.2f}", ha='center', va='bottom')
plt.xticks([0, 1])
plt.suptitle("Out-of-sample prediction(X=1)")
plt.xlabel("romantic")
plt.ylabel("proportion")
sns.despine()
print(y_pred)

[1 1 1 ... 1 1 0]


<Figure size 640x480 with 1 Axes>

在对新结果的预测中，有46%的可能将其预测为0，有54%的可能将其预测为1

In [211]:
coords = {"obs_id": df_new.index}
with pm.Model(coords=coords) as log_model5:
    sex = pm.MutableData("sex", df_new.sex, dims="obs_id")
    anxiety_r=pm.MutableData("anxiety_r", df_new.anxiety_r, dims="obs_id")

    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)            #定义beta_2
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * anxiety_r+beta_2*sex, dims="obs_id")
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=df_new.romantic,dims="obs_id")

    log_model5_trace = pm.sample(draws=5000,                 
                                tune=1000,                  
                                chains=4,                     
                                discard_tuned_samples= True, 
                                random_seed=84735)
    log_model5_ppc = pm.sample_posterior_predictive(log_model5_trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 26 seconds.
Sampling: [y_est]


In [212]:
y_pred = log_model5_ppc.posterior_predictive.y_est.stack(sample = ("chain", "draw")).mean(dim="sample")
y_pred_df = pd.DataFrame(y_pred, columns=["y_pred"])
log_model5_df = pd.DataFrame({"sex": log_model5_ppc.constant_data.sex, "anxiety_r": log_model5_ppc.constant_data.anxiety_r,"romantic": log_model5_ppc.observed_data.y_est})
y_pred_df = pd.concat([y_pred_df, log_model5_df], axis=1)
y_pred_df["romantic_2"] = np.where(np.array(y_pred) >= 0.5, 1, 0)
print(y_pred_df)

confusion_matrix = pd.crosstab(y_pred_df["romantic"], y_pred_df["romantic_2"], 
                              rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

      y_pred  sex  anxiety_r  romantic  romantic_2
0    0.78545    0  -1.975260         1           1
1    0.54995    0  -0.323469         1           1
2    0.68235    1  -1.172962         1           1
3    0.43050    0   0.384442         1           0
4    0.46080    1   0.148472         0           0
..       ...  ...        ...       ...         ...
132  0.43640    1   0.337248         0           0
133  0.56450    0  -0.370663         1           1
134  0.74540    1  -1.644902         1           1
135  0.34400    1   0.903577         0           0
136  0.21790    1   1.894651         0           0

[137 rows x 5 columns]


Predicted,0,1
Actual,,
0,46,23
1,24,44


In [213]:
true_positive = confusion_matrix.at[0, 1]
false_positive = confusion_matrix.at[1, 1]
true_negative = confusion_matrix.at[0, 0]
false_negative = confusion_matrix.at[1, 0]
accuracy = (true_positive + false_negative) /(true_positive + false_positive + true_negative + false_negative)
sensitivity = (true_positive) /(true_positive + false_negative)
specificity = (true_negative) / (true_negative + false_positive)

print("准确性:", accuracy)
print("敏感性:", sensitivity)
print("特异性:", specificity)

准确性: 0.34306569343065696
敏感性: 0.48936170212765956
特异性: 0.5111111111111111


模型对于原数据的预测结果准确性较低为，说明对于模型的整体性适中，依恋焦虑和性别对于预测被试是否会恋爱的准确率为0.34；敏感性为0.49，说明对于依恋焦虑和性别低预测被试进行恋爱的可能性为0.49；特异性为0.51，说明依恋焦虑和性别得分高预测被试不进行恋爱的可能性为0.51。

# 模型比较

In [214]:
df_raw = pd.read_csv('/home/mw/input/bayes20238001/Data_Sum_HPP_Multi_Site_Share.csv')
#进入VCU站点
df = df_raw[df_raw["Site"] == "VCU"]
df = df[["romantic", "sex","anxiety_r"]]
df["romantic"] =  np.where(df['romantic'] == 2, 0, 1)
df["index"] = range(len(df))
df = df.set_index("index")
print(df)

       romantic  sex  anxiety_r
index                          
0             0  2.0   2.132340
1             1  2.0   0.295121
2             0  2.0   1.064189
3             1  1.0  -0.730304
4             0  1.0  -0.473948
...         ...  ...        ...
146           0  1.0   0.166943
147           0  2.0   1.106915
148           1  NaN  -0.773030
149           1  NaN  -2.482070
150           1  NaN  -0.858482

[151 rows x 3 columns]


In [324]:
df.isnull().any(axis=0)
df.loc[:, df.isnull().any(axis=0)]
df['sex'] = df['sex'].fillna(df['sex'].interpolate())
df.fillna(value =1, inplace = True)
df.head()

,romantic,sex,anxiety_r
index,,,
0,0,2.0,2.132340
1,1,2.0,0.295121
2,0,2.0,1.064189
3,1,1.0,-0.730304
4,0,1.0,-0.473948


In [325]:
with pm.Model() as log_model1:
    log_model1.add_coord('obs_id',df.index, mutable=True)
    anxiety_r = pm.MutableData("anxiety_r", df.anxiety_r, dims="obs_id")
    sex= pm.MutableData("sex", df.sex, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id') 

    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)                    
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)

    mu = pm.Deterministic("mu", beta_0 + beta_1 * anxiety_r+beta_2*sex, dims="obs_id")
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")

with pm.Model() as log_model2:
    log_model2.add_coord('obs_id',df.index, mutable=True)
    avoidance = pm.MutableData("anxiety_r", df.anxiety_r, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)                   
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)          
    mu = pm.Deterministic("mu", beta_0 + beta_1 * anxiety_r, dims="obs_id")
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")


with pm.Model() as log_model3:
    log_model3.add_coord('obs_id',df.index, mutable=True)
    sex= pm.MutableData("sex",df.sex, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)                   
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)           
    mu = pm.Deterministic("mu", beta_0 + beta_2 *sex, dims="obs_id")
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")

In [326]:
with log_model1:
    log_model1_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735)
with log_model2:
    log_model2_trace = pm.sample(draws=5000,                  # 使用mcmc方法进行采样，draws为采样次数
                      tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                      chains=4,                     # 链数
                      discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                      idata_kwargs={"log_likelihood": True},
                      random_seed=84735)
with log_model3:
    log_model3_trace = pm.sample(draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                        tune=1000,                   # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                        chains=4,                    # 链数
                        discard_tuned_samples= True, # tune的结果将在采样结束后被丢弃
                        idata_kwargs={"log_likelihood": True},
                        random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 34 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 18 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 26 seconds.


In [327]:
with log_model1:
    log_model1_ppc = pm.sample_posterior_predictive(log_model1_trace, random_seed=84735)
with log_model2:
    log_model2_ppc = pm.sample_posterior_predictive(log_model2_trace, random_seed=84735)
with log_model3:
    log_model3_ppc = pm.sample_posterior_predictive(log_model3_trace, random_seed=84735)

Sampling: [y_est]


Sampling: [y_est]


Sampling: [y_est]


In [328]:
from statistics import median
def MAE(model_ppc):
    pre_x = model_ppc.posterior_predictive["y_est"].stack(sample=("chain", "draw"))
    pre_y_mean = pre_x.mean(axis=1).values

    MAE = pd.DataFrame({
        "romantic_ppc_mean": pre_y_mean,
        "romantic_original": df.romantic
    })

    MAE["pre_error"] = abs(MAE["romantic_original"] -\
                            MAE["romantic_ppc_mean"])

    MAE = median(MAE.pre_error)
    return MAE

log_model1_MAE = MAE(log_model1_ppc)
print(f"模型1 MAE: {log_model1_MAE:.2f}")
log_model2_MAE = MAE(log_model2_ppc)
print(f"模型2 MAE: {log_model2_MAE:.2f}")
log_model3_MAE = MAE(log_model3_ppc)
print(f"模型3 MAE: {log_model3_MAE:.2f}")

模型1 MAE: 0.42
模型2 MAE: 0.42
模型3 MAE: 0.47


模型1,2的预测误差小于模型3，即模型1,2的预测能力强于模型3。